In [45]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
# from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import sklearn.metrics

epochs = 50

In [10]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        try:
            pred = model.predict([tokens, casing,char], verbose=False)[0]   
            pred = pred.argmax(axis=-1) #Predict the classes            
            correctLabels.append(labels)
            predLabels.append(pred)
        except Exception as e:
            continue
        b.update(i)
    return predLabels, correctLabels

In [11]:
def get_sentences(path):
    with open(path,'rb') as f:
        sentences=[]
        sentence=[]
        for line in f:
            splits = [x.decode() for x in line.split()]
            try:
                word = splits[0]
                label = splits[1]
                temp = [word,label]
                sentence.append(temp)
    #             sentence.append(label)
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences

            
        

In [20]:
# adding BIO, no change in labels.
def modify_labels(dataset):
    for sentence in dataset:
        for pos, word in enumerate(sentence):
            label = word[1]
            if label!='0':
                try:
                    if sentence[pos-1][1]!='0' and sentence[pos-1][1][0]=='B' and sentence[pos-1][1][2:] == label:
                        new_label = 'I-'+label
                        word[1] = new_label
                    else:
                        new_label = 'B-'+label
                        word[1] = new_label
                except Exception as e:
                    new_label = 'B-' +label
                    word[1] = new_label
    return dataset

In [21]:
temp = get_sentences('/srv/Resources/SWE/train_corpus.tsv')
test_sentences = get_sentences('/srv/Resources/SWE/test_corpus.tsv')

train_sentences = temp[:6000]
dev_sentences = temp[6000:]

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


6000
885
1780


In [22]:
print(train_sentences[0])

[['I', '0'], ['längden', '0'], ['torde', '0'], ['få', '0'], ['moderna', '0'], ['par', '0'], ['överleva', '0'], ['om', '0'], ['en', '0'], ['part', '0'], ['bara', '0'], ['lever', '0'], ['i', '0'], ['den', '0'], ['lilla', '0'], ['\xadvärlden', '0'], ['och', '0'], ['den', '0'], ['andra', '0'], ['i', '0'], ['den', '0'], ['stora', '0'], ['.', '0']]


In [23]:
train_sentences = modify_labels(train_sentences)
test_sentences = modify_labels(test_sentences)
dev_sentences = modify_labels(dev_sentences)

In [24]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [25]:
print(trainSentences[0])

[['I', ['I'], '0'], ['längden', ['l', 'ä', 'n', 'g', 'd', 'e', 'n'], '0'], ['torde', ['t', 'o', 'r', 'd', 'e'], '0'], ['få', ['f', 'å'], '0'], ['moderna', ['m', 'o', 'd', 'e', 'r', 'n', 'a'], '0'], ['par', ['p', 'a', 'r'], '0'], ['överleva', ['ö', 'v', 'e', 'r', 'l', 'e', 'v', 'a'], '0'], ['om', ['o', 'm'], '0'], ['en', ['e', 'n'], '0'], ['part', ['p', 'a', 'r', 't'], '0'], ['bara', ['b', 'a', 'r', 'a'], '0'], ['lever', ['l', 'e', 'v', 'e', 'r'], '0'], ['i', ['i'], '0'], ['den', ['d', 'e', 'n'], '0'], ['lilla', ['l', 'i', 'l', 'l', 'a'], '0'], ['\xadvärlden', ['\xad', 'v', 'ä', 'r', 'l', 'd', 'e', 'n'], '0'], ['och', ['o', 'c', 'h'], '0'], ['den', ['d', 'e', 'n'], '0'], ['andra', ['a', 'n', 'd', 'r', 'a'], '0'], ['i', ['i'], '0'], ['den', ['d', 'e', 'n'], '0'], ['stora', ['s', 't', 'o', 'r', 'a'], '0'], ['.', ['.'], '0']]


In [26]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token] = True

In [27]:
print(labelSet)

{'I-MISC', '0', 'I-ORG', 'B-PER', 'B-LOC', 'I-LOC', 'I-PER', 'B-MISC', 'B-ORG'}


In [28]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [29]:
print(label2Idx)

{'I-ORG': 2, 'I-LOC': 5, 'B-LOC': 4, 'B-MISC': 7, 'I-PER': 6, 'I-MISC': 0, '0': 1, 'B-PER': 3, 'B-ORG': 8}


In [30]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [21]:
string_words = ' '.join(words.keys())
print(string_words)

grips klartecken artonåring alkoholiserad fantasteri silvergrå förespråkat riktiga improviseras varje vållande Anderssons hyreshus 16.30-tiden 13-åring licenserna Borg 557 fastställa Konfidensindikatorn gallery Dagligen tillsynstillfället Brodys tränarbyten besvärligare tillför Byggandet reservationer objekt tidplanen Superstjärnan fantasi banor arbetsförhållanden startförbud Kirchgasser förskoleåldern kartan Räntan 146.000 makalös visit Morten fåtöljen praktikplatser spretiga modekommentarer omslagspapper Golfaren fara ansvar undre 23-åriga vandrade Söderling Travet dämpade 2:30 kongress budgetproposition bokstavligt 21 vankas hona hjälte distans längtan detaljerade Största Malmqvist styrelsen bordet konstmässorna förbundet önskvärt unik Republikens svenskakunskaper beslutsfattare buffert vittnen följts passa Aktiviteterna snygg sådant anhölls tillfriskna Representation fordonsprogrammet sharialagar vindtät sämsta fortkörningar stundom bevittnade sålde finn förälder På Viberg toppmöte

In [31]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'PADDING_TOKEN': 7, 'numeric': 0, 'contains_digit': 6, 'initialUpper': 3, 'allLower': 1, 'other': 4, 'mainly_numeric': 5, 'allUpper': 2}


In [33]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('swedish_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [29]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[0]))
print(word2Idx)

300
{'klartecken': 3, 'artonåring': 4, 'alkoholiserad': 5, 'fantasteri': 6, 'silvergrå': 7, 'förespråkat': 8, 'riktiga': 9, 'improviseras': 10, 'varje': 11, 'vållande': 12, 'Anderssons': 13, '16.30-tiden': 15, '13-åring': 16, 'licenserna': 17, 'Borg': 18, '557': 19, 'fastställa': 20, 'Konfidensindikatorn': 21, 'gallery': 22, 'Dagligen': 23, 'tillsynstillfället': 24, 'Brodys': 25, 'tränarbyten': 26, 'Vorin': 21808, 'Samaras': 13127, 'reservationer': 30, 'objekt': 31, 'tidplanen': 32, 'Superstjärnan': 33, 'fantasi': 34, 'banor': 35, 'arbetsförhållanden': 36, 'startförbud': 37, 'Kirchgasser': 38, 'förskoleåldern': 39, 'kartan': 40, 'Räntan': 41, 'makalös': 43, 'visit': 44, 'Morten': 45, 'fåtöljen': 46, 'förändringar': 229, 'spretiga': 48, 'modekommentarer': 49, 'omslagspapper': 50, 'Golfaren': 51, 'fara': 52, 'ansvar': 53, 'undre': 54, '23-åriga': 55, 'vandrade': 56, 'Söderling': 57, 'Travet': 58, 'dämpade': 59, '2:30': 60, 'regi': 13132, 'budgetproposition': 62, 'bokstavligt': 63, '21': 

In [34]:
characters=set()
for word in words.keys():
    for char in word:
        characters.add(char)
# print(characters)
characters = set(characters)
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'L': 1, '6': 3, 'e': 2, 's': 4, 'Z': 6, '7': 24, 'z': 8, 'V': 9, ':': 10, 'ë': 13, 'D': 12, 'h': 14, 'k': 0, 'P': 15, 'Y': 16, 'v': 17, '\xad': 27, ',': 19, ';': 46, 'n': 21, 'a': 22, '—': 23, '@': 26, 'ä': 41, 'à': 28, 'B': 25, 'Á': 63, 'S': 33, 'x': 5, '(': 29, 'ô': 34, 'Ö': 48, '*': 35, 'u': 37, '3': 38, 'Q': 7, 'è': 39, 'o': 30, 'f': 40, 'm': 42, '_': 45, '§': 43, 'C': 44, '–': 47, 'F': 49, 'T': 50, 'K': 95, '.': 51, '&': 52, 'R': 53, '?': 54, 'M': 71, 'U': 55, 'c': 56, 'q': 81, 'Ä': 57, 'û': 58, '"': 36, 'Å': 59, 'J': 60, 'í': 61, 'b': 102, 'w': 62, 'A': 11, '’': 69, 'ç': 64, ')': 65, '0': 66, 'é': 67, '8': 68, '+': 70, 'ü': 72, '¾': 73, 'ú': 74, 'ø': 75, '•': 76, 'I': 77, '”': 78, 'ê': 79, 't': 80, 'l': 83, 'G': 84, '…': 85, 'y': 86, 'r': 87, '%': 88, 'O': 89, '4': 90, 'i': 91, 'W': 109, 'd': 92, "'": 93, 'H': 18, '“': 97, 'j': 94, '1': 99, '/': 96, 'g': 32, 'E': 98, '-': 101, 'X': 82, '9': 103, 'ö': 104, '!': 105, '5': 106, 'á': 20, 'å': 100, 'p': 107, '2': 108, 'N': 31}


In [35]:
print(trainSentences[0]) 

[['I', ['I'], '0'], ['längden', ['l', 'ä', 'n', 'g', 'd', 'e', 'n'], '0'], ['torde', ['t', 'o', 'r', 'd', 'e'], '0'], ['få', ['f', 'å'], '0'], ['moderna', ['m', 'o', 'd', 'e', 'r', 'n', 'a'], '0'], ['par', ['p', 'a', 'r'], '0'], ['överleva', ['ö', 'v', 'e', 'r', 'l', 'e', 'v', 'a'], '0'], ['om', ['o', 'm'], '0'], ['en', ['e', 'n'], '0'], ['part', ['p', 'a', 'r', 't'], '0'], ['bara', ['b', 'a', 'r', 'a'], '0'], ['lever', ['l', 'e', 'v', 'e', 'r'], '0'], ['i', ['i'], '0'], ['den', ['d', 'e', 'n'], '0'], ['lilla', ['l', 'i', 'l', 'l', 'a'], '0'], ['\xadvärlden', ['\xad', 'v', 'ä', 'r', 'l', 'd', 'e', 'n'], '0'], ['och', ['o', 'c', 'h'], '0'], ['den', ['d', 'e', 'n'], '0'], ['andra', ['a', 'n', 'd', 'r', 'a'], '0'], ['i', ['i'], '0'], ['den', ['d', 'e', 'n'], '0'], ['stora', ['s', 't', 'o', 'r', 'a'], '0'], ['.', ['.'], '0']]


In [36]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [37]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print(len(train_set[0][2]))

[[5385, 21452, 8078, 24908, 14254, 9290, 1855, 17193, 9169, 1579, 3044, 2909, 403, 19288, 2892, 6423, 11633, 19288, 23332, 403, 19288, 24400, 2725], [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4], array([[77,  0,  0, ...,  0,  0,  0],
       [83, 41, 21, ...,  0,  0,  0],
       [80, 30, 87, ...,  0,  0,  0],
       ...,
       [92,  2, 21, ...,  0,  0,  0],
       [ 4, 80, 30, ...,  0,  0,  0],
       [51,  0,  0, ...,  0,  0,  0]], dtype=int32), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
23
23


In [38]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [65]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 3300        char_input[0][0]                 
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_13 (TimeDistri (None, None, 52, 30) 2730        dropout_7[0][0]                  
__________________________________________________________________________________________________
time_distr

In [66]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch       
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
        print(' ')

Epoch 0/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 1:45 
 
64/65 [============================>.] - ETA: 0s 
Epoch 1/50
 0/65 [..............................] - ETA: 0s 
 
 
64/65 [============================>.] - ETA: 0s 
Epoch 2/50
 0/65 [..............................] - ETA: 0s 
 
 
37/65 [================>.............] - ETA: 5s 


64/65 [============================>.] - ETA: 0s 
Epoch 3/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 2s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 4/50
 0/65 [..............................] - ETA: 0s 
 
 
64/65 [============================>.] - ETA: 0s 
Epoch 5/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 2s 
 
10/65 [===>..........................] - ETA: 5s 


64/65 [============================>.] - ETA: 0s 
Epoch 6/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 2s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 7/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 2s 
 
47/65 [====================>.........] - ETA: 2s 


64/65 [============================>.] - ETA: 0s 
Epoch 8/50
 1/65 [..............................] - ETA: 5s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 9/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 3s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 10/50
 0/65 [..............................] - ETA: 0s 
 
18/65 [=======>......................] - ETA: 12s 


64/65 [============================>.] - ETA: 0s 
Epoch 11/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 2s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 12/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 3s 
 
54/65 [=======================>......] - ETA: 2s 


...

64/65 [============================>.] - ETA: 0s 
Epoch 39/50
64/65 [============================>.] - ETA: 0s 
Epoch 40/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 2s 
 
39/65 [=================>............] - ETA: 7s 


64/65 [============================>.] - ETA: 0s 
Epoch 41/50
 2/65 [..............................] - ETA: 4s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 42/50
 0/65 [..............................] - ETA: 0s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 43/50
 0/65 [..............................] - ETA: 0s 
 
 9/65 [===>..........................] - ETA: 9s 


64/65 [============================>.] - ETA: 0s 
Epoch 44/50
 0/65 [..............................] - ETA: 0s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 45/50
 1/65 [..............................] - ETA: 4s 
 
42/65 [==================>...........] - ETA: 6s 


64/65 [============================>.] - ETA: 0s 
Epoch 46/50
 0/65 [..............................] - ETA: 0s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 47/50
 0/65 [..............................] - ETA: 0s 
 
64/65 [============================>.] - ETA: 0s 
Epoch 48/50
 0/65 [..............................] - ETA: 0s 
 
 2/65 [..............................] - ETA: 3s 
 
13/65 [=====>........................] - ETA: 10s 


64/65 [============================>.] - ETA: 0s 
Epoch 49/50
 0/65 [..............................] - ETA: 0s 
 
64/65 [============================>.] - ETA: 0s 


In [67]:
predLabels, correctLabels = tag_dataset(dev_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(idx2Label)

881/885 [============================>.] - ETA: 0s{0: 'I-MISC', 1: '0', 2: 'I-ORG', 3: 'B-PER', 4: 'B-LOC', 5: 'I-LOC', 6: 'I-PER', 7: 'B-MISC', 8: 'B-ORG'}


In [68]:
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

0.6382171079750092
0.9849802371541502
0.9851296294724232
[0.21428571 0.99458354 0.57142857 0.92195122 0.86227545 0.18181818
 0.95364238 0.43010753 0.61386139]


In [69]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()

print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

1779/1780 [============================>.] - ETA: 0s0.6575258355018065
0.9859031463229719
0.9848379937603535
[0.30555556 0.99543481 0.60465116 0.8792198  0.82768362 0.3902439
 0.93377483 0.30909091 0.67207792]


In [70]:
model.save('swedish_ner_with_char.h5')

In [71]:
print(len(word2Idx.keys()))

26143


In [72]:
print(len(trainSentences)+len(testSentences)+len(devSentences))

8665
